https://www.kaggle.com/c/cian-datafest-2019

Импортируем библиотеки

In [1]:
from glob import glob
from scipy import stats
import pandas as pd

import time
from tqdm import tqdm
from PIL import Image
import numpy as np

Загружаем данные

In [2]:
train_images = glob ('Content/datafest/*/*.jpg')
test_images = glob('Content/test/*.jpg')
train_images[-5:], len(train_images)

(['Content/datafest\\outdoor\\image_999.jpg',
  'Content/datafest\\outdoor\\image_9990.jpg',
  'Content/datafest\\outdoor\\image_9992.jpg',
  'Content/datafest\\outdoor\\image_9994.jpg',
  'Content/datafest\\outdoor\\image_9995.jpg'],
 82982)

Построим фичи с помощью основных статистик

In [3]:
def image_to_features(image):
    features = []
    np_image = np.array(image)
    split1 = np.array_split(np_image, 7, axis = 0)
    split2 = np.array_split(np_image, 7, axis = 1)
    for im_part in [np_image] + split1[:2] + split1[-2:] + split2[:2] + split2[-2:]:
        for i in range(3):
            s = stats.describe(im_part[:,:,i], axis = None)
            features += [s.mean, s.variance, s.minmax[0], s.minmax[1], s.skewness, s.kurtosis]
    return features

Преобразуем картинки в фичи

In [4]:
%%time
door = {"indoor": list(), "outdoor": list()}
for i, path in tqdm(enumerate(train_images), total=len(train_images)):
#     if i%50 != 0: continue
    image = Image.open(path)
    image_class = path.split('/')[-1].split('\\')[1]
    door[image_class].append(image_to_features(image))

100%|████████████████████████████████████████████████████████████████████████████| 82982/82982 [15:36<00:00, 88.59it/s]

Wall time: 15min 36s


In [5]:
test = []
test_ids = []
for i, path in tqdm(enumerate(test_images), total=len(test_images), position=0, leave=True):
#     if i%60 != 0: continue
    image = Image.open(path)
    imageid = int(path.split('/')[-1].split('\\')[-1].replace('.jpg',''))
    test.append(image_to_features(image))
    test_ids.append(imageid)

100%|████████████████████████████████████████████████████████████████████████████| 27660/27660 [04:54<00:00, 93.85it/s]


Построим саму модель

In [12]:
X=np.array(door["indoor"] + door["outdoor"])
y=np.array([0 for i in range(len(door["indoor"]))]+[1 for i in range(len(door["outdoor"]))])

indices = np.random.permutation(X.shape[0])

X_train = X
Y_train = y
x_final = X_train / np.linalg.norm(X_train, axis = 0)
y = Y_train

alpha=100
iterations = 150000
n = x_final.shape[0]
x0 = np.ones((n, 1))
beta = np.random.rand(1,x_final.shape[1])
for i in tqdm(range(iterations)):
    y_pred = np.dot(x_final,beta.T)
    cost = np.sum((y_pred-y.reshape(n,1))**2)/n
    diff = (y_pred-y.reshape(n,1))
    derivative = 2*np.dot(diff.T,x_final)/n
    beta = (beta)-alpha*(derivative)

100%|██████████████████████████████████████████████████████████████████████████| 150000/150000 [37:08<00:00, 67.30it/s]


Применим построенную модель к тестовым данным

In [13]:
X_test = np.array(test)
test_predict = np.dot(X_test / np.linalg.norm(X_test, axis = 0), beta.T)
test_predict = (test_predict-test_predict.min())/(test_predict.max()-test_predict.min())

Сохраним результат

In [14]:
data = pd.DataFrame({'image_number':test_ids}).join(pd.DataFrame(test_predict, columns = ['prob_outdoor'])).sort_values(by="image_number")

In [15]:
data[:10]

,image_number,prob_outdoor
0,0,0.287855
11021,4,0.408603
22186,7,0.586798
1445,14,0.505731
3605,20,0.496292
3961,21,0.402705
4707,23,0.517562
5091,24,0.541140
6595,28,0.640700
7347,30,0.664057


In [16]:
data.to_csv('submission.csv',index=False)

In [17]:
pd.read_csv('submission.csv')

,image_number,prob_outdoor
0,0,0.287855
1,4,0.408603
2,7,0.586798
3,14,0.505731
4,20,0.496292
...,...,...
27655,82972,0.507150
27656,82974,0.721959
27657,82975,0.405581
27658,82977,0.468373
